In [1]:
# imports

import os
import io
import sys
import json
import requests
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess
from huggingface_hub import InferenceClient

In [2]:
# environment

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [3]:
# initialize

openai = OpenAI()
OPENAI_MODEL ="gpt-4o-mini"
QWEN_MODEL="Qwen/Qwen2.5-Coder-32B-Instruct"

In [4]:
client = InferenceClient(
    provider="nebius",
    api_key=os.environ["HF_TOKEN"],
)

In [5]:
system_message = '''
You are a code documentation assistant. Your task is to take any code provided by the user and return the same code with clear, concise, and professional docstrings and inline comments.

Rules:
1. Preserve the original logic, formatting, and indentation of the code.
2. Add docstrings for:
   - Every function and method (describe purpose, parameters, return values, and any side effects).
   - Every class (describe its role, attributes, and relationships if relevant).
   - The overall script/module (describe the main functionality).
3. Add inline comments above or beside complex, non-obvious, or critical lines of code to explain their purpose.
4. Keep explanations simple, clear, and useful for someone reading the code for the first time.
5. Do not remove or alter existing comments unless they are incorrect or redundant.
6. Output only the modified code, without extra explanations outside the code.

'''

In [6]:
def user_prompt_for(code: str) -> str:
    user_prompt = (
        "Please take the following code and add clear, concise, and professional docstrings "
        "for every function, class, and the overall module. Also, add inline comments to explain "
        "non-obvious parts of the logic. Preserve the code structure and formatting.\n\n"
        "Here is the code:\n\n"
    )
    user_prompt += code
    return user_prompt

In [7]:
def messages_for(code):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(code)}
    ]

In [13]:
def optimize_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)

In [14]:
#Quwen
def optimize_qwen(python):
    stream = client.chat.completions.create(model=QWEN_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)

In [15]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [16]:
optimize_gpt(pi)

```python
import time

def calculate(iterations, param1, param2):
    """
    Calculate a mathematical result based on the specified iterations and parameters.
    
    This function performs a series of calculations involving two parameters to 
    derive a result that is adjusted at each step of the iteration.
    
    Parameters:
    iterations (int): The number of iterations to run the calculation.
    param1 (float): A parameter used in the calculations to modify the first term.
    param2 (float): A parameter used in the calculations to modify the second term.
    
    Returns:
    float: The computed result after all iterations.
    """
    result = 1.0  # Initialize the result to the starting value of 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2  # Calculate the first value of j for the current iteration
        result -= (1/j)  # Adjust the result based on the first term
        j = i * param1 + param2  # Calculate the second value of j for the curr

In [17]:
optimize_qwen(pi)

```python
"""
This module calculates a specific mathematical expression based on given parameters
and measures the execution time of the calculation.
"""

import time

def calculate(iterations, param1, param2):
    """
    Calculate the result of a specific mathematical series.

    Parameters:
    iterations (int): The number of iterations to perform in the series.
    param1 (float): The multiplier used in the calculation of the series terms.
    param2 (float): The subtractor/adder used in the calculation of the series terms.

    Returns:
    float: The result of the series calculation.

    This function iteratively computes a series where each term is the sum of two fractions,
    one with a negative sign and one with a positive sign, using the provided parameters.
    """
    result = 1.0  # Initialize the result to 1.0 as per the problem statement
    for i in range(1, iterations+1):
        j = i * param1 - param2  # Calculate the denominator for the negative fraction
        

In [18]:
python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [19]:
optimize_gpt(python_hard)

```python
"""
This module implements a Linear Congruential Generator (LCG) to produce pseudo-random numbers 
and calculates the maximum subarray sum from a generated list of random integers. It computes 
the total maximum subarray sum over 20 iterations of random number generation.
"""

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    """
    Generate an infinite sequence of pseudo-random numbers using the Linear Congruential Generator (LCG) algorithm.

    Parameters:
    seed (int): The initial seed value for the LCG.
    a (int): The multiplier used in the LCG formula (default is 1664525).
    c (int): The increment used in the LCG formula (default is 1013904223).
    m (int): The modulus used in the LCG formula (default is 2^32).

    Yields:
    int: The next pseudo-random number in the sequence.

    Side Effects:
    This function will yield an infinite number of values and can be used in a loop.
    """
    value = seed
    while True:
        # Calculate the next value in 

In [20]:
optimize_qwen(python_hard)

```python
"""
This module provides functions to generate pseudo-random numbers using a Linear Congruential Generator (LCG)
and to calculate the maximum subarray sum from these numbers over multiple runs.
"""

# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    """
    Generate an infinite sequence of pseudo-random numbers using the Linear Congruential Generator (LCG) algorithm.

    Parameters:
    - seed (int): The initial seed value for the generator.
    - a (int): The multiplier factor of the generator (default is 1664525).
    - c (int): The increment factor of the generator (default is 1013904223).
    - m (int): The modulus factor of the generator (default is 2^32).

    Yields:
    - int: The next pseudo-random number in the sequence.
    """
    value = seed
    while True:
        # Update the value using the LCG formula and yield the result
        value = (a * value + c) % m
        yield value

def max_subarray_sum(n, seed, min_

In [23]:
def stream_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```python\n','').replace('```','')

In [24]:
def stream_qwen(python):    
    stream = client.chat.completions.create(model=QWEN_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```python\n','').replace('```','')

In [25]:
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Qwen":
        result = stream_qwen(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far 

In [26]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Uncomment code:", lines=10, value=python_hard)
        cpp = gr.Textbox(label="Comment code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Qwen"], label="Select model", value="GPT")
        convert = gr.Button("Add comments")

    convert.click(optimize, inputs=[python, model], outputs=[cpp])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
